In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# activate R magic
%load_ext rpy2.ipython

In [ ]:
#not work
%%R
install.packages("/content/MicroMacroMultilevel_0.4.0.tar.gz", repos=NULL, type="source", lib="/usr/local/lib/R/site-library")

In [ ]:
#not work
%%R
install.packages("feather")
install.packages("MicroMacroMultilevel")
library(feather)
library("dplyr")
library("MicroMacroMultilevel")

R[write to console]: Installing package into ‘/root/R/x86_64-pc-linux-gnu-library/4.1’
(as ‘lib’ is unspecified)

R[write to console]: trying URL 'https://cran.rstudio.com/src/contrib/feather_0.3.5.tar.gz'

R[write to console]: Content type 'application/x-gzip'
R[write to console]:  length 73812 bytes (72 KB)

R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: 

In [ ]:
%%R
x <- read_feather("/content/drive/My Drive/Dissertation/airbnb/cultural distance/revisions/revision2/lcm/xdata_full.feather")
z <- read_feather("/content/drive/My Drive/Dissertation/airbnb/cultural distance/revisions/revision2/lcm/zdata_full.feather")
x.gid <- x[['gid']]
x.data <- select(x, cult_dst_6, interaction)
#x.data$nothing <- sample(100, size = nrow(x.data), replace = TRUE)
z.gid <- z[['gid']]

data <- z
epsilon <- 0.0000001
percent_perf <- (30-data$availability_30)/30
z$perf <- ifelse(percent_perf == 0, percent_perf+epsilon, percent_perf)
z$perf <- ifelse(percent_perf == 1, percent_perf-epsilon, z$perf)
z$logit_perf <- log(z$perf/(1-z$perf))

#add seasonality
z$jan <- ifelse(substr(z$yyyymm, 5, 6)=="01", 1, 0)
z$feb <- ifelse(substr(z$yyyymm, 5, 6)=="02", 1, 0)
z$mar <- ifelse(substr(z$yyyymm, 5, 6)=="03", 1, 0)
z$apr <- ifelse(substr(z$yyyymm, 5, 6)=="04", 1, 0)
z$may <- ifelse(substr(z$yyyymm, 5, 6)=="05", 1, 0)
z$jun <- ifelse(substr(z$yyyymm, 5, 6)=="06", 1, 0)
z$jul <- ifelse(substr(z$yyyymm, 5, 6)=="07", 1, 0)
z$aug <- ifelse(substr(z$yyyymm, 5, 6)=="08", 1, 0)
z$sep <- ifelse(substr(z$yyyymm, 5, 6)=="09", 1, 0)
z$oct <- ifelse(substr(z$yyyymm, 5, 6)=="10", 1, 0)
z$nov <- ifelse(substr(z$yyyymm, 5, 6)=="11", 1, 0)

y <- z[['logit_perf']]
z.data <- select(z, gid, host_is_superhost, number_of_reviews, price, host_listings_count, jan, feb, mar, apr, may, jun, jul, aug, sep, oct, nov)
colnames(z.data)[1] <- "group.id"
rm(x, z, data, epsilon, percent_perf)

In [ ]:
%%R
results = adjusted.predictors(x.data, z.data, x.gid, z.gid)

In [ ]:
%%R
model.formula = as.formula(y ~ BLUP.cult_dst_6 + BLUP.interaction + host_is_superhost + number_of_reviews
                           + price + host_listings_count + jan + feb + mar + apr + may + jun + jul + aug + sep + oct + nov)
model.output = micromacro.lm(model.formula, results$adjusted.group.means, y, results$unequal.groups)
micromacro.summary(model.output)

Call:
micromacro.lm( y ~ BLUP.cult_dst_6 + BLUP.interaction + host_is_superhost + number_of_reviews + price + host_listings_count, ...)
    
Residuals:
       Min        1Q      Median        3Q      Max
 -18.94295 -1.190486 -0.09232311 0.9000749 16.31791

    
Coefficients:
                         Estimate Uncorrected S.E. Corrected S.E.    df
(Intercept)          0.2412389703     0.0628683557   0.0877028649 55539
BLUP.cult_dst_6     -0.0162007625     0.0457222162   0.0510914812 55539
BLUP.interaction     0.0314244107     0.0335209056   0.0347291324 55539
host_is_superhost    0.4216437442     0.0394895321   0.0410296262 55539
number_of_reviews    0.0045468309     0.0002229835   0.0002287574 55539
price               -0.0006945238     0.0001700913   0.0005355778 55539
host_listings_count  0.0009315329     0.0010552649   0.0008922749 55539
                             t     Pr(>|t|)           r
(Intercept)          2.7506396 5.949830e-03 0.011670920
BLUP.cult_dst_6     -0.3170932 7.511

In [ ]:
%%R
saveRDS(model.output, "/content/drive/My Drive/m1.rds")

# **run full data with TPU**

In [ ]:
%%R
#save(y, file="/content/drive/My Drive/y.RData")
#save(results, file="/content/drive/My Drive/temp.RData")
load("/content/drive/My Drive/temp.RData")
load("/content/drive/My Drive/y.RData")

In [ ]:
%%R
micromacro.lm = function(model, adjusted.predictors, y, unequal.groups=NULL) {
  data = cbind(adjusted.predictors,y)
  uncorrected.output = lm( model, data)
  model.terms = attr(terms.formula(model), "term.labels")
  model.variables = model.terms[!grepl(":",model.terms)]
  interaction.terms = model.terms[grepl(":",model.terms)]
  if (length(interaction.terms)>0) {
    interactions = matrix( NA, nrow=dim(data)[1], ncol=length(interaction.terms) )
    for ( i in 1:length(interaction.terms) ) {
      interactions[,i] = apply( data[strsplit(interaction.terms,":")[[i]]], 1, prod )
    }
    u = cbind(rep(1,dim(data)[1]),as.matrix(cbind(data[model.variables], interactions)))
  } else {u = cbind(rep(1,dim(data)[1]),as.matrix(cbind(data[model.variables])))}
  e = uncorrected.output$residuals
  p = solve(t(u) %*% u)
  h = diag(u %*% p %*% t(u))
  d = e^2/(1-h)
  v = p %*% t(u) %*% diag(d) %*% u %*% p
  se = sqrt(diag(v))
  estim = summary(uncorrected.output)$coefficients[,1:2]
  t.stats = estim[,1]/se
  p.values = pt(-abs(t.stats),rep(uncorrected.output$df.residual,length(model.terms)+1))*2 # two-sided
  effect.sizes = sqrt( t.stats**2/(t.stats**2+uncorrected.output$df.residual) )
  estim1 = cbind(estim,se,rep(uncorrected.output$df.residual,length(model.terms)+1),t.stats,p.values,effect.sizes)
  dimnames(estim1)[[2]] = c("Estimate","Uncorrected S.E.","Corrected S.E.","df","t","Pr(>|t|)","r")
  # print( estim )
  if (is.null(unequal.groups)){ # default (null) is same group size
    t.stats2 = estim[,1]/estim[,2]
    p.values2 = pt(-abs(t.stats2),rep(uncorrected.output$df.residual,length(model.terms)+1))*2
    effect.sizes2 = sqrt( t.stats2**2/(t.stats2**2+uncorrected.output$df.residual) )
    estim2 = cbind(estim,rep(uncorrected.output$df.residual,length(model.terms)+1),t.stats2,p.values2,effect.sizes2)
    dimnames(estim2)[[2]] = c("Estimate","S.E.","df","t","Pr(>|t|)","r")
    report = list("statistics" = estim2, "rsquared" = summary(uncorrected.output)$r.squared, "rsquared.adjusted" = summary(uncorrected.output)$adj.r.squared, "residuals" = uncorrected.output$residuals, "fitted.values" = uncorrected.output$fitted.values, "fstatistic" = summary(uncorrected.output)$fstatistic, "model.formula" = model)
  } else if (isTRUE(unequal.groups)) { # true when group sizes are unequal
    report = list("statistics" = estim1, "rsquared" = summary(uncorrected.output)$r.squared, "rsquared.adjusted" = summary(uncorrected.output)$adj.r.squared, "residuals" = uncorrected.output$residuals, "fitted.values" = uncorrected.output$fitted.values, "fstatistic" = summary(uncorrected.output)$fstatistic, "model.formula" = model)
  } else { # true when group sizes are the same
    t.stats2 = estim[,1]/estim[,2]
    p.values2 = pt(-abs(t.stats2),rep(uncorrected.output$df.residual,length(model.terms)+1))*2
    effect.sizes2 = sqrt( t.stats2**2/(t.stats2**2+uncorrected.output$df.residual) )
    estim2 = cbind(estim,rep(uncorrected.output$df.residual,length(model.terms)+1),t.stats2,p.values2,effect.sizes2)
    dimnames(estim2)[[2]] = c("Estimate","S.E.","df","t","Pr(>|t|)","r")
    report = list("statistics" = estim2, "rsquared" = summary(uncorrected.output)$r.squared, "rsquared.adjusted" = summary(uncorrected.output)$adj.r.squared, "residuals" = uncorrected.output$residuals, "fitted.values" = uncorrected.output$fitted.values, "fstatistic" = summary(uncorrected.output)$fstatistic, "model.formula" = model)
  }
  return(report)
}

In [ ]:
%%R
micromacro.summary = function(model.output) {
  cat(noquote("Call:"))
  cat("\n")
  cat(noquote(sprintf("micromacro.lm( %s %s %s, ...)", gettext(model.output$model.formula)[2], gettext(model.output$model.formula)[1], gettext(model.output$model.formula)[3])))
  cat("\n")
  cat(noquote("    "))
  cat("\n")
  # summarize the residuals
  cat(noquote("Residuals:"))
  cat("\n")
  residual.summary = cbind(min(model.output$residuals), quantile(model.output$residuals, 0.25), quantile(model.output$residuals, 0.5), quantile(model.output$residuals, 0.75), max(model.output$residuals))
  dimnames(residual.summary)[[1]] = ""
  dimnames(residual.summary)[[2]] = c("Min","1Q","Median","3Q","Max")
  print(noquote(residual.summary))
  cat("\n")
  cat(noquote("    "))
  cat("\n")
  # summarize the coefficients
  cat(noquote("Coefficients:"))
  cat("\n")
  print(noquote(model.output$statistics))
  cat("\n")
  cat(noquote("---"))
  cat("\n")
  cat(noquote(sprintf("Residual standard error: %s on %s degrees of freedom", round(sd(model.output$residuals), digits = 5), length(model.output$residuals)-dim(model.output$statistics)[1])))
  cat("\n")
  cat(noquote(sprintf("Multiple R-squared: %s, Adjusted R-squared: %s", round(model.output$rsquared, digits = 10), round(model.output$rsquared.adjusted, digits = 10))))
  cat("\n")
  cat(noquote(sprintf("F-statistic: %s on %s and %s DF, p-value: %s", round(model.output$fstatistic[1], digits = 5), model.output$fstatistic[2], model.output$fstatistic[3], round(pf(model.output$fstatistic[1], model.output$fstatistic[2], model.output$fstatistic[3], lower.tail=FALSE), digits = 5))))
}

In [ ]:
%%R
model.formula = as.formula(y ~ BLUP.cult_dst_6 + BLUP.interaction + host_is_superhost + number_of_reviews
                           + price + host_listings_count + jan + feb + mar + apr + may + jun + jul + aug + sep + oct + nov)
model.output = micromacro.lm(model.formula, results$adjusted.group.means, y, results$unequal.groups)
micromacro.summary(model.output)

Call:
micromacro.lm( y ~ BLUP.cult_dst_6 + BLUP.interaction + host_is_superhost + number_of_reviews + price + host_listings_count + jan + feb + mar + apr + may + jun + jul + aug + sep + oct + nov, ...)
    
Residuals:
       Min        1Q     Median        3Q      Max
 -19.69008 -1.361168 -0.3585987 0.7144378 20.37806

    
Coefficients:
                        Estimate Uncorrected S.E. Corrected S.E.    df
(Intercept)         -0.484338806     0.1087925275   0.1034392264 64941
BLUP.cult_dst_6     -0.057695511     0.0402957841   0.0435559238 64941
BLUP.interaction     0.046251607     0.0294136418   0.0301070418 64941
host_is_superhost    0.405405705     0.0344528132   0.0364932812 64941
number_of_reviews    0.005047335     0.0001997767   0.0002120960 64941
price               -0.000834269     0.0001547901   0.0006119852 64941
host_listings_count -0.002234491     0.0009462110   0.0008593094 64941
jan                 -3.054546747     0.1241431606   0.1473646189 64941
feb                 -

# **final code**



In [4]:
%%R

library("dplyr")
load("/content/drive/My Drive/Dissertation/airbnb/cultural distance x quality signaling/lvmlm/201810 onward/xdata.RData")
load("/content/drive/My Drive/Dissertation/airbnb/cultural distance x quality signaling/lvmlm/201810 onward/zdata.RData")
x.gid <- x[['gid']]
x.data <- select(x, cult_dst_6, interaction)
#x.data$nothing <- sample(100, size = nrow(x.data), replace = TRUE)
z.gid <- z[['gid']]

data <- z
epsilon <- 0.0000001
percent_perf <- (30-data$availability_30)/30
z$perf <- ifelse(percent_perf == 0, percent_perf+epsilon, percent_perf)
z$perf <- ifelse(percent_perf == 1, percent_perf-epsilon, z$perf)
z$logit_perf <- log(z$perf/(1-z$perf))

#add seasonality
z$jan <- ifelse(substr(z$yyyymm, 5, 6)=="01", 1, 0)
z$feb <- ifelse(substr(z$yyyymm, 5, 6)=="02", 1, 0)
z$mar <- ifelse(substr(z$yyyymm, 5, 6)=="03", 1, 0)
z$apr <- ifelse(substr(z$yyyymm, 5, 6)=="04", 1, 0)
z$may <- ifelse(substr(z$yyyymm, 5, 6)=="05", 1, 0)
z$jun <- ifelse(substr(z$yyyymm, 5, 6)=="06", 1, 0)
z$jul <- ifelse(substr(z$yyyymm, 5, 6)=="07", 1, 0)
z$aug <- ifelse(substr(z$yyyymm, 5, 6)=="08", 1, 0)
z$sep <- ifelse(substr(z$yyyymm, 5, 6)=="09", 1, 0)
z$oct <- ifelse(substr(z$yyyymm, 5, 6)=="10", 1, 0)
z$nov <- ifelse(substr(z$yyyymm, 5, 6)=="11", 1, 0)

y <- z[['logit_perf']]
z.data <- select(z, gid, host_is_superhost, price, number_of_reviews, review_scores_rating, jan, feb, mar, apr, may, jun, jul, aug, sep, oct, nov)
colnames(z.data)[1] <- "group.id"
rm(x, z, data, epsilon, percent_perf)

In [9]:
%%R
results = adjusted.predictors(x.data, z.data, x.gid, z.gid)

In [19]:
%%R
model.formula = as.formula(y ~ BLUP.cult_dst_6 + BLUP.interaction + host_is_superhost 
                          # + price + review_scores_rating + number_of_reviews
                          )
model.output = micromacro.lm(model.formula, results$adjusted.group.means, y, results$unequal.groups)
micromacro.summary(model.output)

Call:
micromacro.lm( y ~ BLUP.cult_dst_6 + host_is_superhost, ...)
    
Residuals:
       Min        1Q     Median       3Q      Max
 -16.97031 -1.148097 0.01487802 1.052054 15.87914

    
Coefficients:
                     Estimate Uncorrected S.E. Corrected S.E.    df         t
(Intercept)        0.33252487       0.06432597     0.07099994 42534  4.683453
BLUP.cult_dst_6   -0.02631538       0.04897851     0.05604706 42534 -0.469523
host_is_superhost  0.51969752       0.04567232     0.04570058 42534 11.371792
                      Pr(>|t|)           r
(Intercept)       2.829522e-06 0.022703150
BLUP.cult_dst_6   6.386983e-01 0.002276605
host_is_superhost 6.384103e-30 0.055055614

---
Residual standard error: 4.70483 on 42534 degrees of freedom
Multiple R-squared: 0.0030403596, Adjusted R-squared: 0.0029934814
F-statistic: 64.85651 on 2 and 42534 DF, p-value: 0

In [16]:
%%R
saveRDS(model.output, "/content/drive/My Drive/m1.rds")

In [17]:
%%R
model.formula = as.formula(y ~ BLUP.cult_dst_6 + BLUP.interaction + host_is_superhost
                          # + price + review_scores_rating + number_of_reviews
                           + jan + feb + mar + apr + may + jun + jul + aug + sep + oct + nov
                           )
model.output = micromacro.lm(model.formula, results$adjusted.group.means, y, results$unequal.groups)
micromacro.summary(model.output)

Call:
micromacro.lm( y ~ BLUP.cult_dst_6 + BLUP.interaction + host_is_superhost + jan + feb + mar + apr + may + jun + jul + aug + sep + oct + nov, ...)
    
Residuals:
       Min        1Q     Median        3Q      Max
 -19.02796 -1.295239 -0.2026173 0.9204053 19.44473

    
Coefficients:
                     Estimate Uncorrected S.E. Corrected S.E.    df           t
(Intercept)       -0.20889121       0.11404509     0.08299245 42522  -2.5169904
BLUP.cult_dst_6    0.02789921       0.04986783     0.05621141 42522   0.4963264
BLUP.interaction  -0.03196246       0.03800242     0.03978347 42522  -0.8034107
host_is_superhost  0.54986540       0.04310742     0.04319142 42522  12.7308942
jan               -3.10428415       0.13073886     0.14760155 42522 -21.0315146
feb               -0.09774187       0.13220144     0.08642363 42522  -1.1309624
mar               -0.58011985       0.12641963     0.09442685 42522  -6.1435901
apr               -1.24465827       0.12088656     0.10413186 42522 -1

In [18]:
%%R
saveRDS(model.output, "/content/drive/My Drive/m1-1.rds")

In [6]:
#' Calculating the Adjusted Group Means of Individual-Level Variables in a Micro-Macro Multilevel Situation
#'
#' As the prerequisite step of fitting a micro-macro multilevel model, this function calculates the adjusted group means of individual-level predictors in an unbiased way.
#'
#' @param x.data an N-by-p data frame of individual-level predictors, where N denotes the total number of individuals and p denotes the number of individual-level predictors. Must contain no NAs.
#' @param z.data a G-by-q data frame of group-level predictors, where G denotes the total number of groups and q denotes the number of group-level predictors. Must contain no NAs.
#' @param x.gid an array or an N-by-1 numeric matrix of each individual's group ID. The order corresponds to the individuals in x.data. Duplicates expected.
#' @param z.gid an array or a G-by-1 numeric matrix of Group ID. The order corresponds to the groups in z.data. All group IDs should be unique (i.e., no duplicates allowed).
#' @return adjusted.group.means a G-by-(p+q+1) numeric matrix that contains p adjusted group means of the individual-level variables from x.data, q group-level predictors from z.data, and unique group IDs.
#' @return unequal.groups a boolean variable. TRUE = group size is different across groups; FALSE = group size is the same across groups.
#' @return group.size a G-by-2 data frame that displays unique group IDs and the corresponding group sizes.
#' @keywords multilevel modeling, micro-macro, using individual-level predictors to predict group-level outcome variables
#' @details
#'
#' To date, most multilevel methodologies can only unbiasedly model macro-micro multilevel situations, wherein group-level predictors (e.g., city temperature)
#' are used to predict an individual-level outcome variable (e.g., citizen personality). In contrast, this R package enables researchers to model micro-macro situations,
#' wherein individual-level (micro) predictors (and other group-level predictors) are used to predict a group-level (macro) outcome variable in an unbiased way.
#'
#' To conduct micro-macro multilevel modeling with the current package, one must first compute the adjusted group means with the function \code{\link{adjusted.predictors}}.
#' This is because in micro-macro multilevel modeling, it is statistically biased to directly regress the group-level outcome variable on the unadjusted group means of individual-level predictors (Croon & van Veldhoven, 2007).
#' Instead, one should use the best linear unbiased predictors (BLUP) of the group means (i.e., the adjusted group means), which is conveniently computed by \code{\link{adjusted.predictors}}.
#'
#' Once produced by \code{\link{adjusted.predictors}}, the adjusted group means can be used as one of the inputs of the \code{\link{micromacro.lm}} function, which reports estimation results and inferential statistics of the micro-macro multilevel model of interest.
#' Importantly, \code{\link{adjusted.predictors}} also reports whether group size is the same across all groups, which is a critical dummy input of the \code{\link{micromacro.lm}} function.
#'
#' @seealso \code{\link{micromacro.lm}} for fitting the micro-macro multilevel linear model of interest.
#' @author Jackson G. Lu, Elizabeth Page-Gould, Nancy R. Xu (maintainer, nancyranxu@gmail.com).
#' @references
#' Akinola, M., Page-Gould, E., Mehta, P. H., & Lu, J. G. (2016). Collective hormonal profiles predict group performance. Proceedings of the National Academy of Sciences, 113 (35), 9774-9779.
#'
#' Croon, M. A., & van Veldhoven, M. J. (2007). Predicting group-level outcome variables from variables measured at the individual level: A latent variable multilevel model. Psychological Methods, 12(1), 45-57.
#' @example R/Examples/adjustedpredictorsExample2.r
#' @importFrom stats cov lm pf pt quantile sd terms.formula var
#' @export
%%R
adjusted.predictors = function( x.data, z.data , x.gid, z.gid) {
  x.data = as.data.frame(x.data)
  z.data = as.data.frame(z.data)
  # sort individual-level data based on group ID
  x = x.data[order(x.gid),]
  # sort group-level data
  z = z.data[order(z.gid),]
  # number of individuals per group
  ng = as.numeric(unlist(tapply(rep(1,dim(x)[1]),x.gid,sum)))
  #1a: Parameters pertaining to distribution of Z
  mu_z = apply( z, 2, mean ) # 1-by-q
  sigma_zz = var( z ) # q-by-q
  #1b: matrix of observed means
  mu_x = apply( x, 2, mean ) # 1-by-p
  mu_xg = apply( x, 2, function(x) {tapply(x, x.gid, mean)} ) # G by p, mean by group, the unadjusted mean
  gid = x.gid
  x.data2 = cbind(x, gid)
  gid = z.gid
  mu_xg2 = cbind(mu_xg, gid)
  x.data.means = merge( x.data2, mu_xg2, by="gid" ) # merge group means with individual level data
  #1c: Covariance matrix of Z and X
  sigma_xiz = cov( mu_xg, z ) # p by q, between-group covariance
  sigma_zxi = t( sigma_xiz ) # q by p, transpose
  #1d: Between- and within-group covariance matrices
  SSA = sum(ng)*apply(mu_xg,1,function(x){x-mu_x})%*%t(apply(mu_xg,1,function(x){x-mu_x})) # p by p
  MSA = SSA/(dim(x)[1]-dim(z)[1]) # p by p
  SSE = matrix( 0, nrow=dim(x)[2], ncol=dim(x)[2]) # p by p
  for (k in 1:(dim(x.data.means)[1])) {
    increment = (dim(x.data.means)[2]-1)/2 # should be the same as the unique individual-level variables
    SSE_i = as.numeric( unlist( x.data.means[k,2:(dim(x.data.means)[2]-increment)] - x.data.means[k,(dim(x.data.means)[2]-increment+1):dim(x.data.means)[2]] ) )%*%t(as.numeric( unlist( x.data.means[k,2:(dim(x.data.means)[2]-increment)] - x.data.means[k,(dim(x.data.means)[2]-increment+1):dim(x.data.means)[2]] ) ))
    SSE = SSE + SSE_i
  }
  MSE = SSE/( dim(z)[1] - 1 )
  sigma_vv = MSE # p by p
  sigma_xixi = ((dim(x)[1]*(dim(z)[1] - 1))/(dim(x)[1]**2-sum(ng**2)))*(MSA-MSE) # p by p
  #2: Adjusted group means
  adjusted.means = matrix( NA, nrow=dim(z)[1], ncol=dim(x)[2] ) # G by p
  for (g in 1:dim(z)[1]) {
    if (dim(z)[2]>0) {
      W_g1 = solve(sigma_xixi + sigma_vv/as.numeric(ng)[g] + sigma_xiz%*%solve(sigma_zz)%*%sigma_zxi)%*%(sigma_xixi + sigma_xiz%*%solve(sigma_zz)%*%sigma_zxi)
      W_g2 = solve(sigma_zz)%*%sigma_zxi%*%(diag(dim(x)[2])-W_g1)
      if (dim(as.matrix(z[g,]-mu_z,nrow=1))[1] == 1){
        addx = as.matrix(z[g,]-mu_z,nrow=1)
      } else {
        addx = t(as.matrix(z[g,]-mu_z,nrow=1))
      }
      adjusted.means[g,] = t(mu_x)%*%(diag(dim(x)[2])-W_g1)+t(mu_xg[g,])%*%W_g1+addx%*%W_g2
    } else {
      W_g1 = solve(sigma_xixi + sigma_vv/as.numeric(ng)[g])%*%(sigma_xixi)
      adjusted.means[g,] = t(mu_x)%*%(diag(dim(x)[2])-W_g1)+t(mu_xg[g,])%*%W_g1
    }
  }
  adjusted.group.means = as.data.frame( cbind( adjusted.means, z, z.gid) )
  names(adjusted.group.means) = c( paste( "BLUP.", names(x), sep="" ), names(z), "gid")

  # assign boolean variable to unequal.groups which returns TRUE when groups sizes are different, FALSE otherwise
  unequal.groups = isTRUE(min(ng) != max(ng))

  group.size = as.data.frame(cbind(z.gid, ng))
  report =  list("unequal.groups" = unequal.groups, "adjusted.group.means" = adjusted.group.means, "group.size" = group.size)
  return(report)
}






In [7]:
#' Fitting Micro-Macro Multilevel Linear Models
#'
#' After computing the adjusted group means of individual-level predictors by \code{\link{adjusted.predictors}}, use \code{\link{micromacro.lm}} for estimation results and inferential statistics.
#' @param model a linear regression model formula, e.g., as.formula(y ~ x1 + x2 ... + xm).
#' @param adjusted.predictors a G-by-m data frame, where column variables are group-level predictors and the adjusted group means of individual-level predictors were computed by the \code{\link{adjusted.predictors}} function. G denotes the number of groups and m denotes the number of predictors in the model.
#' @param y an array or a G-by-1 numeric matrix that corresponds to the group-level outcome variable in the model.
#' @param unequal.groups an optional boolean variable automatically reported by the \code{\link{adjusted.predictors}} function. TRUE = group size is different across groups; FALSE = group size is the same across groups. Default is FALSE (same group size).
#' @return statistics a summary reports standard inferential statistics on linear regression,
#' e.g., "Estimate", coefficient estimates; "Uncorrected S.E."/"S.E.", OLS standard errors;
#' "Corrected S.E.", heteroskedasticity-consistent standard errors; "df", degree of freedom; "t", Student t statistics;
#' "Pr(>|t|)", two-sided p-value; "r", effect size.
#' @return rsquared r squared.
#' @return rsquared.adjusted adjusted r squared.
#' @return residuals residuals from the model.
#' @return fitted.values fitted values from the model.
#' @return fstatistic F statistics of the model.
#' @return model.formula model formula.
#' @keywords multi-level modeling, micro-macro, prediciting group-level variables from individual-level variables, different group sizes
#' @details
#'
#' To date, most multilevel methodologies can only unbiasedly model macro-micro multilevel situations, wherein group-level predictors (e.g., city temperature)
#' are used to predict an individual-level outcome variable (e.g., citizen personality). In contrast, this R package enables researchers to model micro-macro situations,
#' wherein individual-level (micro) predictors (and other group-level predictors) are used to predict a group-level (macro) outcome variable in an unbiased way.
#'
#' To conduct micro-macro multilevel modeling with the current package, one must first compute the adjusted group means with the function \code{\link{adjusted.predictors}}.
#' This is because in micro-macro multilevel modeling, it is statistically biased to directly regress the group-level outcome variable on the unadjusted group means of individual-level predictors (Croon & van Veldhoven, 2007).
#' Instead, one should use the best linear unbiased predictors (BLUP) of the group means (i.e., the adjusted group means), which is conveniently computed by \code{\link{adjusted.predictors}}.
#'
#' Once produced by \code{\link{adjusted.predictors}}, the adjusted group means can be used as one of the inputs of the \code{\link{micromacro.lm}} function, which reports estimation results and inferential statistics of the micro-macro multilevel model of interest.
#'
#' If group size is the same across all groups (i.e., unequal.groups = FALSE), then OLS standard errors are reported and used to determine the inferential statistics in this micro-macro model.
#' If group size is different across groups (i.e., unequal.groups = TRUE), however, then the heteroscedasticity-consistent standard errors are reported and used determine the inferential statistics in this micro-macro model (White, 1980).
#'
#' @seealso \code{\link{adjusted.predictors}} for calculating the adjusted group means of the individual-level predictors, and \code{\link{micromacro.summary}} for a friendly output summary table.
#' @author Jackson G. Lu, Elizabeth Page-Gould, & Nancy R. Xu (maintainer, nancyranxu@gmail.com).
#' @references
#' Akinola, M., Page-Gould, E., Mehta, P. H., & Lu, J. G. (2016). Collective hormonal profiles predict group performance. Proceedings of the National Academy of Sciences, 113 (35), 9774-9779.
#'
#' Croon, M. A., & van Veldhoven, M. J. (2007). Predicting group-level outcome variables from variables measured at the individual level: A latent variable multilevel model. Psychological Methods, 12(1), 45-57.
#'
#' White, H. (1980). A heteroskedasticity-consistent covariance estimator and a direct test of heteroskedasticity. Econometrica, 48, 817-838.
#' @example R/Examples/micromacrolmExample2.r
#' @importFrom stats cov lm pf pt quantile sd terms.formula var
#' @export

%%R
micromacro.lm = function(model, adjusted.predictors, y, unequal.groups=NULL) {
  data = cbind(adjusted.predictors,y)
  uncorrected.output = lm( model, data)
  model.terms = attr(terms.formula(model), "term.labels")
  model.variables = model.terms[!grepl(":",model.terms)]
  interaction.terms = model.terms[grepl(":",model.terms)]
  if (length(interaction.terms)>0) {
    interactions = matrix( NA, nrow=dim(data)[1], ncol=length(interaction.terms) )
    for ( i in 1:length(interaction.terms) ) {
      interactions[,i] = apply( data[strsplit(interaction.terms,":")[[i]]], 1, prod )
    }
    u = cbind(rep(1,dim(data)[1]),as.matrix(cbind(data[model.variables], interactions)))
  } else {u = cbind(rep(1,dim(data)[1]),as.matrix(cbind(data[model.variables])))}
  e = uncorrected.output$residuals
  p = solve(t(u) %*% u)
  h = diag(u %*% p %*% t(u))
  d = e^2/(1-h)
  v = p %*% t(u) %*% diag(d) %*% u %*% p
  se = sqrt(diag(v))
  estim = summary(uncorrected.output)$coefficients[,1:2]
  t.stats = estim[,1]/se
  p.values = pt(-abs(t.stats),rep(uncorrected.output$df.residual,length(model.terms)+1))*2 # two-sided
  effect.sizes = sqrt( t.stats**2/(t.stats**2+uncorrected.output$df.residual) )
  estim1 = cbind(estim,se,rep(uncorrected.output$df.residual,length(model.terms)+1),t.stats,p.values,effect.sizes)
  dimnames(estim1)[[2]] = c("Estimate","Uncorrected S.E.","Corrected S.E.","df","t","Pr(>|t|)","r")
  # print( estim )
  if (is.null(unequal.groups)){ # default (null) is same group size
    t.stats2 = estim[,1]/estim[,2]
    p.values2 = pt(-abs(t.stats2),rep(uncorrected.output$df.residual,length(model.terms)+1))*2
    effect.sizes2 = sqrt( t.stats2**2/(t.stats2**2+uncorrected.output$df.residual) )
    estim2 = cbind(estim,rep(uncorrected.output$df.residual,length(model.terms)+1),t.stats2,p.values2,effect.sizes2)
    dimnames(estim2)[[2]] = c("Estimate","S.E.","df","t","Pr(>|t|)","r")
    report = list("statistics" = estim2, "rsquared" = summary(uncorrected.output)$r.squared, "rsquared.adjusted" = summary(uncorrected.output)$adj.r.squared, "residuals" = uncorrected.output$residuals, "fitted.values" = uncorrected.output$fitted.values, "fstatistic" = summary(uncorrected.output)$fstatistic, "model.formula" = model)
  } else if (isTRUE(unequal.groups)) { # true when group sizes are unequal
    report = list("statistics" = estim1, "rsquared" = summary(uncorrected.output)$r.squared, "rsquared.adjusted" = summary(uncorrected.output)$adj.r.squared, "residuals" = uncorrected.output$residuals, "fitted.values" = uncorrected.output$fitted.values, "fstatistic" = summary(uncorrected.output)$fstatistic, "model.formula" = model)
  } else { # true when group sizes are the same
    t.stats2 = estim[,1]/estim[,2]
    p.values2 = pt(-abs(t.stats2),rep(uncorrected.output$df.residual,length(model.terms)+1))*2
    effect.sizes2 = sqrt( t.stats2**2/(t.stats2**2+uncorrected.output$df.residual) )
    estim2 = cbind(estim,rep(uncorrected.output$df.residual,length(model.terms)+1),t.stats2,p.values2,effect.sizes2)
    dimnames(estim2)[[2]] = c("Estimate","S.E.","df","t","Pr(>|t|)","r")
    report = list("statistics" = estim2, "rsquared" = summary(uncorrected.output)$r.squared, "rsquared.adjusted" = summary(uncorrected.output)$adj.r.squared, "residuals" = uncorrected.output$residuals, "fitted.values" = uncorrected.output$fitted.values, "fstatistic" = summary(uncorrected.output)$fstatistic, "model.formula" = model)
  }
  return(report)
}


In [8]:
#' Summarizing the Micro-Macro Multilevel Linear Model Fitting Results
#'
#' After fitting a micro-macro multilevel model, this function produces a user-friendly summary table of the results.
#'
#' @param model.output the output of \code{\link{micromacro.lm}} which contains model results and model specifications.
#' @return table a summary table.
#' @keywords multilevel modeling, micro-macro, using individual-level predictors to predict group-level outcome variables
#' @details
#'
#' To date, most multilevel methodologies can only unbiasedly model macro-micro multilevel situations, wherein group-level predictors (e.g., city temperature)
#' are used to predict an individual-level outcome variable (e.g., citizen personality). In contrast, this R package enables researchers to model micro-macro situations,
#' wherein individual-level (micro) predictors (and other group-level predictors) are used to predict a group-level (macro) outcome variable in an unbiased way.
#'
#' To conduct micro-macro multilevel modeling with the current package, one must first compute the adjusted group means with the function \code{\link{adjusted.predictors}}.
#' This is because in micro-macro multilevel modeling, it is statistically biased to directly regress the group-level outcome variable on the unadjusted group means of individual-level predictors (Croon & van Veldhoven, 2007).
#' Instead, one should use the best linear unbiased predictors (BLUP) of the group means (i.e., the adjusted group means), which is conveniently computed by \code{\link{adjusted.predictors}}.
#'
#' Once produced by \code{\link{adjusted.predictors}}, the adjusted group means can be used as one of the inputs of the \code{\link{micromacro.lm}} function, which reports estimation results and inferential statistics of the micro-macro multilevel model of interest.
#'
#' If group size is the same across all groups (i.e., unequal.groups = FALSE), then OLS standard errors are reported and used to determine the inferential statistics in this micro-macro model.
#' If group size is different across groups (i.e., unequal.groups = TRUE), however, then the heteroscedasticity-consistent standard errors are reported and used determine the inferential statistics in this micro-macro model (White, 1980).
#'
#' \code{\link{micromacro.summary}} produces a detailed summary on the model fitting and specifications, given the outputs of \code{\link{micromacro.lm}}.
#'
#' @seealso \code{\link{micromacro.lm}} for fitting the micro-macro multilevel linear model of interest.
#' @author Jackson G. Lu, Elizabeth Page-Gould, Nancy R. Xu (maintainer, nancyranxu@gmail.com).
#' @references
#' Akinola, M., Page-Gould, E., Mehta, P. H., & Lu, J. G. (2016). Collective hormonal profiles predict group performance. Proceedings of the National Academy of Sciences, 113 (35), 9774-9779.
#'
#' Croon, M. A., & van Veldhoven, M. J. (2007). Predicting group-level outcome variables from variables measured at the individual level: a latent variable multilevel model. Psychological methods, 12(1), 45-57.
#' @example R/Examples/micromacrolmExample2.r
#' @importFrom stats cov lm pf pt quantile sd terms.formula var
#' @export

%%R

micromacro.summary = function(model.output) {
  cat(noquote("Call:"))
  cat("\n")
  cat(noquote(sprintf("micromacro.lm( %s %s %s, ...)", gettext(model.output$model.formula)[2], gettext(model.output$model.formula)[1], gettext(model.output$model.formula)[3])))
  cat("\n")
  cat(noquote("    "))
  cat("\n")
  # summarize the residuals
  cat(noquote("Residuals:"))
  cat("\n")
  residual.summary = cbind(min(model.output$residuals), quantile(model.output$residuals, 0.25), quantile(model.output$residuals, 0.5), quantile(model.output$residuals, 0.75), max(model.output$residuals))
  dimnames(residual.summary)[[1]] = ""
  dimnames(residual.summary)[[2]] = c("Min","1Q","Median","3Q","Max")
  print(noquote(residual.summary))
  cat("\n")
  cat(noquote("    "))
  cat("\n")
  # summarize the coefficients
  cat(noquote("Coefficients:"))
  cat("\n")
  print(noquote(model.output$statistics))
  cat("\n")
  cat(noquote("---"))
  cat("\n")
  cat(noquote(sprintf("Residual standard error: %s on %s degrees of freedom", round(sd(model.output$residuals), digits = 5), length(model.output$residuals)-dim(model.output$statistics)[1])))
  cat("\n")
  cat(noquote(sprintf("Multiple R-squared: %s, Adjusted R-squared: %s", round(model.output$rsquared, digits = 10), round(model.output$rsquared.adjusted, digits = 10))))
  cat("\n")
  cat(noquote(sprintf("F-statistic: %s on %s and %s DF, p-value: %s", round(model.output$fstatistic[1], digits = 5), model.output$fstatistic[2], model.output$fstatistic[3], round(pf(model.output$fstatistic[1], model.output$fstatistic[2], model.output$fstatistic[3], lower.tail=FALSE), digits = 5))))
}
